In [1]:
import numpy as np
import pandas as pd
from mylibs import tools, functions as f

In [2]:
%matplotlib qt

In [5]:
exp = tools.Experiment()
exp.load_csv(*tools.input_path())
tools.temporal_plot(exp)
exp.set_info(
    compound=input('Compound: '),
    rho=float(input('Rho: ')),
    w=float(input('W mass: ')),
)

In [7]:
exp = tools.configurate_data(exp)

In [8]:
tools.temperature_plot(
    exp,
    title='Viscosity',
    xlabel='Temperature [C]',
    ylabel='Viscosity [cP]',
    interactive=True,
)


In [9]:
exp.apply(f.C_to_K)
exp.apply(f.nu_D)
exp.apply(f.linearize)
exp.group_filter(f.iqr_filter)
exp.apply(f.delinearize)
exp.apply(f.K_to_C)
print('Filtered')

Filtered


In [10]:
tools.temperature_plot(
    exp,
    title='Diffusion',
    xlabel='Temperature [C]',
    ylabel='D [m2/s]',
    interactive=True,
)

In [11]:
info, result, func = tools.regress(exp)
exp.set_info(**info)
x = np.linspace(13, 42, 100) + 273.15
ols_res = tools.Experiment(
    pd.DataFrame({
        'x': x, 'y': func(x), 'time': x * 0
    }),
    'interpolated',
)

ols_res.apply(f.K_to_C);

In [12]:
exp.log

[('csv loaded',
  'D:\\Works\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.csv'),
 ('initial_filter', {'time': [20.0, inf], 'y': [20.0, 300.0], 'x': (12, 42)}),
 ('C_to_K', []),
 ('nu_D', []),
 ('linearize', []),
 ('iqr_filter', []),
 ('delinearize', []),
 ('K_to_C', [])]

In [11]:
tmp=exp.copy()

In [13]:
exp2= exp.copy()
ols_res2 = ols_res.copy()
exp2.apply(f.C_to_K)
ols_res2.apply(f.C_to_K)
exp2.apply(f.linearize)
ols_res2.apply(f.linearize)

tools.comparation_plot(
    exp2,
    ols_res2,
    title='OLS_Linear',
    xlabel='Temperature',
    ylabel='D',
    interactive=True,
)

In [14]:
tools.comparation_plot(
    exp,
    ols_res,
    title='OLS_Diffusion',
    xlabel='Temperature [C]',
    ylabel='D [m2/s]',
    interactive=True,
)

In [22]:
exp.save_hdf5()

# Load

In [20]:
import os
for i in range(9):

    exp = tools.Experiment()
    path = f'D:\Works\Diplom-work\Experiments\OCM_viscosity\OCM{i}'
    file = [file for file in os.listdir(path) if '.hdf5' in file][0]
    exp.load_hdf5(*tools.input_path(f'{path}\{file}'))
    exp.log.pop()
    info, result, func = tools.regress(exp)
    exp.set_info(**info)
    # os.remove(f'{path}\{file}')
    exp.save_hdf5(f'{path}\{file}')

NotADirectoryError: ``D:\Works\Diplom-work\Experiments\OCM_viscosity\OCM0\data_OCM0_up.hdf5`` is not a directory

In [28]:
with pd.HDFStore('D:\Works\Diplom-work\Experiments\OCM_viscosity\OCM0\data_OCM0_up.hdf5') as file:
    log = file.get_storer('data').attrs.log 
    info= file.get_storer('data').attrs.info 

In [29]:
log

[('csv loaded',
  'D:\\Works\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.csv'),
 ('initial_filter', {'time': [20.0, inf], 'y': [20.0, 300.0], 'x': (12, 42)}),
 ('C_to_K', []),
 ('nu_D', []),
 ('linearize', []),
 ('iqr_filter', []),
 ('delinearize', []),
 ('K_to_C', [])]

In [33]:

exp2 =tools.Experiment()
exp2.load_hdf5(*tools.input_path())
exp2.log

[('csv loaded',
  'D:\\WORKS\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.csv'),
 ('initial_filter', {'time': [0.0, inf], 'y': [40.0, 350.0], 'x': (12, 42)}),
 ('C_to_K', []),
 ('nu_D', []),
 ('linearize', []),
 ('iqr_filter', []),
 ('delinearize', []),
 ('K_to_C', []),
 ('hdf5 loaded',
  'D:\\Works\\Diplom-work\\Experiments\\OCM_viscosity\\OCM0\\data_OCM0_up.hdf5')]

In [34]:
exp2.info

{'compound': 'ocm',
 'rho': 1.72,
 'w': 0.0,
 'D0': 58053559827657.89,
 'd_D0': 2256959362103.371,
 'd_E': 95.76658562539887,
 'f_statistic': 3029495.381638645,
 'r2': 0.9951642727866552,
 'E': 59372.593499571696,
 'dD0': 2256959362103.371,
 'dE': 95.76658562539887}

In [36]:
tools.temporal_base_plot(exp2)

KeyboardInterrupt: Interrupted by user